In [41]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

File ‘bbc_text_cls.csv’ already there; not retrieving.



In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GlobalMaxPooling1D
from tensorflow.keras.layers import LSTM,GRU,SimpleRNN,Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [44]:
df=pd.read_csv('bbc_text_cls.csv')

In [45]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [46]:
df['target']=df['labels'].astype('category').cat.codes
df['target']

0       0
1       0
2       0
3       0
4       0
       ..
2220    4
2221    4
2222    4
2223    4
2224    4
Name: target, Length: 2225, dtype: int8

In [47]:
k=df['target'].max() +1
k

5

In [48]:
from sklearn.model_selection import train_test_split
df_train,df_test=train_test_split(df,test_size=0.3,random_state=0)

In [49]:
MAX_VOCAB_SIZE=2000                    #converting sentences to sequences
tokenizer=Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df_train['text'])
sequences_train=tokenizer.texts_to_sequences(df_train['text'])
sequences_test=tokenizer.texts_to_sequences(df_test['text'])

In [50]:
word2idx=tokenizer.word_index #word to integer mapping
v=len(word2idx)
v

27489

In [51]:
data_train=pad_sequences(sequences_train)  #padding sequences to get a N X T matrix
print(data_train.shape)  #shape of data_train tensor
T=data_train.shape[1]
data_test=pad_sequences(sequences_test)
print(data_test.shape)

(1557, 2831)
(668, 3477)


In [52]:
D=20
i=Input(shape=(T,))
x=Embedding(v+1,D)(i)
x=LSTM(32,return_sequences=True)(x)
x=GlobalMaxPooling1D()(x)
x=Dense(k)(x)

model=Model(i,x)

In [53]:
model.compile(optimizer='adam',metrics=['accuracy'],loss=SparseCategoricalCrossentropy(from_logits=True))

In [56]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 2831)]            0         
                                                                 
 embedding_2 (Embedding)     (None, 2831, 20)          549800    
                                                                 
 lstm_2 (LSTM)               (None, 2831, 32)          6784      
                                                                 
 global_max_pooling1d_2 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 556,749
Trainable params: 556,749
Non-trainable params: 0
_____________________________________________________

In [55]:
r=model.fit(data_train,df_train['target'],validation_data=(data_test,df_test['target']),epochs=13)

Epoch 1/13
 5/49 [==>...........................] - ETA: 2:59 - loss: 1.6104 - accuracy: 0.2000

KeyboardInterrupt: ignored

In [40]:
plt.plot(r.history['loss'],label='training loss')
plt.plot(r.history['val_loss'],label='value loss')
plt.legend();

NameError: ignored

In [ ]:
plt.plot(r.history['accuracy'],label='train accuracy')
plt.plot(r.history['val_accuracy'],label='value accuracy')
plt.legend();